In [1]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from sklearn.model_selection import train_test_split
from openai import OpenAI
from tqdm import tqdm
import ast
import time
import random
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report

In [3]:
# Calculate the Mean Absolute Error (MAE) for each model with respect to the oracle scor
df = pd.read_csv('Training.csv')

mae_claude = df['mae_with_oracle_claude'].mean()
mae_openai = df['mae_with_oracle_openai'].mean()
mae_llama = df['mae_with_oracle_llama'].mean()

mae_claude, mae_openai, mae_llama

(0.2959827357237716, 0.13728419654714474, 0.8454515272244356)

In [4]:
df.head()

,user_id,question_id,answer_claude,justification_claude,score_claude,answer_openai,justification_openai,score_openai,answer_llama,justification_llama,...,alignment_with_oracle_llama,mae_with_oracle_llama,filled_prompt,answer_judge,justification_judge,score_judge,filled_prompt_x,filled_prompt_y,question_cand_answer,tokenized_questions
0,349,9,"{'expertise': {'justification': ""The candidate...",The candidate demonstrates a good understandin...,4,"{'expertise': {'justification': ""The candidate...",The candidate uses some technical terminology ...,2,"{'expertise': {'justification': ""The candidate...",The candidate demonstrates a good understandin...,...,True,0,\n\n### Evaluator Background\n\nYou are an exp...,"{'expertise': {'justification': ""The candidate...",The candidate demonstrates a solid understandi...,4,NaN,NaN,"Interviewer: Certainly, I was asking about the...","[[101, 4357, 2121, 1024, 5121, 1010, 1045, 200..."
1,411,1,{'expertise': {'justification': 'The candidate...,The candidate demonstrates a good understandin...,4,"{'expertise': {'justification': ""The candidate...",The candidate accurately uses technical termin...,4,"{'expertise': {'justification': ""The candidate...",The candidate struggles to accurately use tech...,...,False,2,\n\n### Evaluator Background\n\nYou are an exp...,"{'expertise': {'justification': ""The candidate...",The candidate demonstrates a solid understandi...,4,NaN,NaN,Interviewer: Great! I see from your resume tha...,"[[101, 4357, 2121, 1024, 2307, 999, 1045, 2156..."
2,825,4,"{'expertise': {'justification': ""The candidate...",The candidate demonstrates a good understandin...,4,"{'expertise': {'justification': ""The candidate...",The candidate uses technical terminology such ...,4,"{'expertise': {'justification': ""The candidate...",The candidate demonstrates a good understandin...,...,False,1,\n\n### Evaluator Background\n\nYou are an exp...,"{'expertise': {'justification': ""The candidate...",The candidate demonstrates a solid understandi...,4,NaN,NaN,"Interviewer: I'm sorry to interrupt, but could...","[[101, 4357, 2121, 1024, 1045, 1005, 1049, 337..."
3,765,7,"{'expertise': {'justification': ""The candidate...",The candidate demonstrates a good understandin...,4,"{'expertise': {'justification': ""The candidate...",The candidate uses technical terminology such ...,4,{'expertise': {'justification': 'The candidate...,The candidate demonstrates some technical know...,...,False,2,\n\n### Evaluator Background\n\nYou are an exp...,"{'expertise': {'justification': ""The candidate...",The candidate demonstrates a solid understandi...,4,NaN,NaN,Interviewer: Great! Let's dive into the case s...,"[[101, 4357, 2121, 1024, 2307, 999, 2292, 1005..."
4,773,6,"{'expertise': {'justification': ""The candidate...",The candidate demonstrates a good understandin...,4,"{'expertise': {'justification': ""The candidate...",The candidate uses technical terminology such ...,4,"{'expertise': {'justification': ""The candidate...",The candidate demonstrates some technical know...,...,False,2,\n\n### Evaluator Background\n\nYou are an exp...,"{'expertise': {'justification': ""The candidate...",The candidate demonstrates a solid understandi...,4,NaN,NaN,Interviewer: Your use of in-memory caching wit...,"[[101, 4357, 2121, 1024, 2115, 2224, 1997, 199..."


In [5]:
df['score_openai'].value_counts(), df['score_claude'].value_counts(), df['score_llama'].value_counts(), df['oracle_score'].value_counts()

(score_openai
 4    2620
 3    1723
 1     969
 2     597
 5     115
 Name: count, dtype: int64,
 score_claude
 4    2563
 3    1197
 2     846
 5     539
 1     509
 0     370
 Name: count, dtype: int64,
 score_llama
 2    3620
 3    1277
 1     919
 4     201
 0       6
 5       1
 Name: count, dtype: int64,
 oracle_score
 4    2435
 3    1785
 1     913
 2     891
 Name: count, dtype: int64)

# As there are mislabbeled '5' scores in the oracle_score column, I'm considering openai_score as the ground_truth scores for the model training because of the low MAE

In [6]:
print(df['score_openai'].value_counts(normalize=True))
print(f"Unique values in score_openai: {df['score_openai'].unique()}")

score_openai
4    0.434927
3    0.286023
1    0.160857
2    0.099104
5    0.019090
Name: proportion, dtype: float64
Unique values in score_openai: [2 4 3 5 1]


In [ ]:
# def balanced_sample(df, class_column, n_samples=30):
#     """
#     Take n_samples from each class in the dataframe.
#     If a class has fewer than n_samples, take all samples from that class.

#     Parameters:
#     df (pd.DataFrame): The input dataframe
#     class_column (str): The name of the column containing the class labels
#     n_samples (int): The number of samples to take from each class

#     Returns:
#     pd.DataFrame: A new dataframe with balanced classes
#     """

#     # Get unique classes
#     classes = df[class_column].unique()

#     # Initialize an empty list to store sampled dataframes
#     sampled_dfs = []

#     for cls in classes:
#         # Get all samples of the current class
#         class_df = df[df[class_column] == cls]

#         # If the class has fewer samples than n_samples, take all of them
#         # Otherwise, take a random sample of n_samples
#         if len(class_df) <= n_samples:
#             sampled_dfs.append(class_df)
#         else:
#             sampled_dfs.append(class_df.sample(n_samples, random_state=42))

#     # Concatenate all sampled dataframes
#     balanced_df = pd.concat(sampled_dfs, ignore_index=True)

#     # Shuffle the final dataframe
#     balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

#     return balanced_df

In [ ]:
# balanced_df = balanced_sample(df, 'score_openai', n_samples=115)

# # Check the new class distribution
# print(balanced_df['score_openai'].value_counts())

score_openai
4    115
3    115
5    115
2    115
1    115
Name: count, dtype: int64


### Create a subset of 600 samples (500 for training, 100 for val/test), stratified by openai_score


In [7]:
# Function to create a stratified sample based on score_opwnai
def stratified_sample(data, n_samples, strata_col='score_openai'):
    return data.groupby(strata_col, group_keys=False).apply(
        lambda x: x.sample(min(len(x), max(1, int(n_samples * len(x) / len(data)))))
    )

In [17]:
subset = stratified_sample(df, 600)

In [18]:
print(subset['score_openai'].value_counts(normalize=True))
print(f"Unique values in score_openai: {subset['score_openai'].unique()}")

score_openai
4    0.435511
3    0.286432
1    0.160804
2    0.098827
5    0.018425
Name: proportion, dtype: float64
Unique values in score_openai: [1 2 3 4 5]


In [19]:
print(subset['score_openai'].value_counts())

score_openai
4    260
3    171
1     96
2     59
5     11
Name: count, dtype: int64


In [20]:
# Select only the required columns
subset = subset[['question_cand_answer', 'score_openai', 'justification_openai']]

In [22]:
subset['score_openai'].head(10)

,score_openai
5217,1
3263,1
3052,1
3849,1
3176,1
4355,1
3920,1
5401,1
3778,1
3538,1


In [23]:
# Shuffle the subset
subset = subset.sample(frac=1, random_state=42).reset_index(drop=True)

In [24]:
subset['score_openai'].value_counts()

,count
score_openai,
4,260
3,171
1,96
2,59
5,11


In [25]:
print(subset['score_openai'].value_counts(normalize=True))
print(f"Unique values in score_openai: {subset['score_openai'].unique()}")
print("\nFirst 20 openai scores after shuffling:")
print(subset['score_openai'].head(20))

score_openai
4    0.435511
3    0.286432
1    0.160804
2    0.098827
5    0.018425
Name: proportion, dtype: float64
Unique values in score_openai: [2 4 1 3 5]

First 20 openai scores after shuffling:
0     2
1     4
2     2
3     1
4     4
5     3
6     1
7     5
8     1
9     4
10    1
11    3
12    3
13    4
14    3
15    3
16    1
17    4
18    4
19    1
Name: score_openai, dtype: int64


### Now comes the process of making the jsonl data for the fine-tuning use case


In [26]:
# Define the system and user role templates
system_role = """You are an expert technical hiring manager evaluating interviews of candidates for various technical roles. Your task is to critically assess the candidate's responses, looking for exceptional talent. Your evaluation should be based on a specific rubric focusing on the candidate's expertise.

Evaluation Criteria for Expertise:
1. Accuracy and depth of technical terminology usage
2. Demonstration of deep knowledge through concrete examples
3. Appropriateness of technicality level for the given context

Scoring Rubric:
1 = Minimal technical knowledge; rarely uses technical terms accurately
2 = Basic understanding; some technical language but often inaccurate or misapplied
3 = Competent knowledge; generally correct terminology, working understanding of concepts
4 = Strong expertise; accurate use of technical language, in-depth knowledge with minor gaps
5 = Exceptional mastery; flawless technical accuracy, comprehensive understanding with insightful examples

Your task is to provide a score (1-5) and a detailed justification for your evaluation. Base your assessment solely on the evidence provided in the candidate's response. Consider how well they explain concepts, the accuracy of their explanations, and the depth of understanding demonstrated through examples or applications they discuss.
"""

user_role_template = """Evaluate the following interview response for technical expertise:

Interviewer: {interviewer_question}
Interviewee: {interviewee_response}
"""

In [27]:
# Function to split the question_cand_answer into interviewer and interviewee parts
def split_qa(qa_string):
    parts = qa_string.split('Interviewee:', 1)
    interviewer = parts[0].split('Interviewer:', 1)[1].strip()
    interviewee = parts[1].strip() if len(parts) > 1 else ''
    return interviewer, interviewee

In [28]:
# Function to create a JSONL entry
def create_jsonl_entry(row):
    interviewer_q, interviewee_a = split_qa(row['question_cand_answer'])
    user_content = user_role_template.format(
        interviewer_question=interviewer_q,
        interviewee_response=interviewee_a
    )

    assistant_content = json.dumps({
        'expertise': {
            'justification': row['justification_openai'],
            'score': int(row['score_openai'])
        }
    })

    messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_content},
        {"role": "assistant", "content": assistant_content}
    ]

    return json.dumps({"messages": messages})

In [29]:
# Create JSONL entries
jsonl_entries = subset.apply(create_jsonl_entry, axis=1)

In [30]:
# Save to JSONL file
with open('interview_evaluations_4.jsonl', 'w') as f:
    for entry in jsonl_entries:
        f.write(f"{entry}\n")

print("JSONL file 'interview_evaluations_4.jsonl' has been created.")

JSONL file 'interview_evaluations_4.jsonl' has been created.


In [31]:
# Display the first entry as an example
print("\nExample of the first JSONL entry:")
print(jsonl_entries.iloc[0])


Example of the first JSONL entry:
{"messages": [{"role": "system", "content": "You are an expert technical hiring manager evaluating interviews of candidates for various technical roles. Your task is to critically assess the candidate's responses, looking for exceptional talent. Your evaluation should be based on a specific rubric focusing on the candidate's expertise.\n\nEvaluation Criteria for Expertise:\n1. Accuracy and depth of technical terminology usage\n2. Demonstration of deep knowledge through concrete examples\n3. Appropriateness of technicality level for the given context\n\nScoring Rubric:\n1 = Minimal technical knowledge; rarely uses technical terms accurately\n2 = Basic understanding; some technical language but often inaccurate or misapplied\n3 = Competent knowledge; generally correct terminology, working understanding of concepts\n4 = Strong expertise; accurate use of technical language, in-depth knowledge with minor gaps\n5 = Exceptional mastery; flawless technical ac

### Split jsonl file into train, test and validation set

In [32]:
# Split into train+val and test sets
train_val, test = train_test_split(jsonl_entries, test_size=0.15, stratify=subset['score_openai'], random_state=42)

# Split train+val into train and validation sets
train, val = train_test_split(train_val, test_size=0.15, stratify=subset.loc[train_val.index, 'score_openai'], random_state=42)


In [33]:
# Save each set to a separate file
for data, filename in zip([train, val, test], ['train_openai.jsonl', 'val_openai.jsonl', 'test_openai.jsonl']):
    with open(filename, 'w') as f:
        for entry in data:
            f.write(f"{entry}\n")

print(f"Created files: train_openai.jsonl ({len(train)} entries), val_openai.jsonl ({len(val)} entries), test_openai.jsonl ({len(test)} entries)")

Created files: train_openai.jsonl (430 entries), val_openai.jsonl (77 entries), test_openai.jsonl (90 entries)


# **Trying the finetuned gpt-4o model:**

---


## Specs:
- Trained on 430 samples
- Temperature set to 0
- Max tokens: 1000
- JSON object already included in the training

In [34]:
# Initialize the OpenAI client
client = OpenAI(api_key='your_api_key')

In [35]:
available_models = client.models.list()
for model in available_models:
    print(model.id)

text-embedding-3-small
gpt-4-0125-preview
text-embedding-ada-002
whisper-1
dall-e-2
tts-1-hd-1106
tts-1-1106
tts-1-hd
dall-e-3
babbage-002
gpt-4-turbo-preview
gpt-4-1106-preview
gpt-4o-2024-05-13
gpt-3.5-turbo-0613
gpt-4
gpt-3.5-turbo
gpt-4o-mini
gpt-3.5-turbo-1106
gpt-3.5-turbo-16k
gpt-3.5-turbo-16k-0613
gpt-4o-mini-2024-07-18
chatgpt-4o-latest
gpt-4-vision-preview
gpt-3.5-turbo-instruct-0914
gpt-3.5-turbo-0125
gpt-4o-2024-08-06
gpt-4o
gpt-4-0613
gpt-3.5-turbo-instruct
gpt-3.5-turbo-0301
tts-1
gpt-4-turbo-2024-04-09
gpt-4-1106-vision-preview
davinci-002
gpt-4-turbo
text-embedding-3-large
ada:ft-scispace:citation-classification-trial-2022-10-28-12-39-45
ada:ft-scispace:citation-classification-trial-2-2022-10-28-13-57-47
ada:ft-scispace:citation-intent-2022-10-29-11-00-34
ft:gpt-4o-2024-08-06:scispace:two-step-finetuning:A0PW1g9O
ft:gpt-4o-2024-08-06:scispace:two-step-finetuning-final:A0WTYMJt:ckpt-step-430
ft:gpt-4o-2024-08-06:scispace:two-step-finetuning-final:A0WTZ40p
ft:gpt-4o-2024-

In [37]:
# Load the test set
with open('test_openai.jsonl', 'r') as f:
    test_data = [json.loads(line) for line in f]

In [38]:
# Function to extract content from the message format
def extract_content(messages, role):
    for message in messages:
        if message['role'] == role:
            return message['content']
    return None

In [39]:
# Function to create user role prompt
def create_user_prompt(user_content):
    # Extract interviewer and interviewee text
    parts = user_content.split("Interviewee:", 1)
    interviewer_text = parts[0].split("Interviewer:", 1)[1].strip()
    interviewee_text = parts[1].strip() if len(parts) > 1 else ""

    # Format the user role prompt
    user_role = f"""Evaluate the following interview response for technical expertise:

    Interviewer: {interviewer_text}
    Interviewee: {interviewee_text}
    """
    return user_role

In [40]:
# Function to make API call with exponential backoff
def make_api_call(system_content, user_content, max_retries=5):
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model ="ft:gpt-4o-2024-08-06:scispace:two-step-finetuning-final:A0WTYMJt:ckpt-step-430",
                messages=[
                    {
                        "role": "system",
                        "content": [{"type": "text", "text": system_content}]
                    },
                    {
                        "role": "user",
                        "content": [{"type": "text", "text": user_content}]
                    }
                ],
                temperature=0,
                max_tokens=1000,
                top_p=1,
                frequency_penalty=0,
                presence_penalty=0,
                response_format={"type": "text"}
            )
            return response.choices[0].message.content
        except Exception as e:
            if attempt < max_retries - 1:
                sleep_time = (2 ** attempt) + (random.random() * 0.1)
                print(f"API call failed. Retrying in {sleep_time:.2f} seconds...")
                time.sleep(sleep_time)
            else:
                print(f"API call failed after {max_retries} attempts. Error: {str(e)}")
                return None

In [41]:
# Lists to store results
predictions = []
actual_scores = []
actual_justifications = []

# Process each example in the test set
for example in tqdm(test_data, desc="Processing test data"):
    system_content = extract_content(example['messages'], 'system')
    user_content = extract_content(example['messages'], 'user')
    actual_assistant_content = extract_content(example['messages'], 'assistant')

    # Create formatted user prompt
    formatted_user_prompt = create_user_prompt(user_content)

    # Make API call
    api_response = make_api_call(system_content, formatted_user_prompt)

    if api_response:
        predictions.append(api_response)

        # Extract actual score and justification
        try:
            actual_content = ast.literal_eval(actual_assistant_content)
            actual_scores.append(actual_content['expertise']['score'])
            actual_justifications.append(actual_content['expertise']['justification'])
        except:
            print(f"Error parsing actual content: {actual_assistant_content}")
            actual_scores.append(None)
            actual_justifications.append(None)
    else:
        predictions.append(None)
        actual_scores.append(None)
        actual_justifications.append(None)

Processing test data: 100%|██████████| 90/90 [03:04<00:00,  2.05s/it]


### Creating a dataframe for storing results from the API

In [42]:
# Create a DataFrame with results
results = pd.DataFrame({
    'Predicted': predictions,
    'Actual_Score': actual_scores,
    'Actual_Justification': actual_justifications
})

In [43]:
results

,Predicted,Actual_Score,Actual_Justification
0,"{""expertise"": {""justification"": ""The candidate...",3,The candidate uses technical terminology such ...
1,"{""expertise"": {""justification"": ""The candidate...",1,The candidate's response is incomplete and doe...
2,"{""expertise"": {""justification"": ""The candidate...",5,The candidate uses technical terminology such ...
3,"{""expertise"": {""justification"": ""The candidate...",2,The candidate uses some technical terminology ...
4,"{""expertise"": {""justification"": ""The candidate...",1,The candidate asked for the question to be rep...
...,...,...,...
85,"{""expertise"": {""justification"": ""The candidate...",4,The candidate uses technical terminology such ...
86,"{""expertise"": {""justification"": ""The candidate...",4,The candidate uses technical terminology such ...
87,"{""expertise"": {""justification"": ""The candidate...",4,The candidate uses technical terminology such ...
88,"{""expertise"": {""justification"": ""The candidate...",3,The candidate uses technical terminology such ...


In [44]:
# Save results to CSV
results.to_csv('model_test_results.csv', index=False)

print("Testing completed. Results saved to 'model_test_results.csv'")

Testing completed. Results saved to 'model_test_results.csv'


In [45]:
# Basic evaluation (assuming we can extract predicted scores)
def extract_predicted_score(prediction):
    try:
        pred_dict = ast.literal_eval(prediction)
        return pred_dict['expertise']['score']
    except:
        return None

results['Predicted_Score'] = results['Predicted'].apply(extract_predicted_score)

In [46]:
results

,Predicted,Actual_Score,Actual_Justification,Predicted_Score
0,"{""expertise"": {""justification"": ""The candidate...",3,The candidate uses technical terminology such ...,4
1,"{""expertise"": {""justification"": ""The candidate...",1,The candidate's response is incomplete and doe...,1
2,"{""expertise"": {""justification"": ""The candidate...",5,The candidate uses technical terminology such ...,4
3,"{""expertise"": {""justification"": ""The candidate...",2,The candidate uses some technical terminology ...,2
4,"{""expertise"": {""justification"": ""The candidate...",1,The candidate asked for the question to be rep...,1
...,...,...,...,...
85,"{""expertise"": {""justification"": ""The candidate...",4,The candidate uses technical terminology such ...,4
86,"{""expertise"": {""justification"": ""The candidate...",4,The candidate uses technical terminology such ...,3
87,"{""expertise"": {""justification"": ""The candidate...",4,The candidate uses technical terminology such ...,4
88,"{""expertise"": {""justification"": ""The candidate...",3,The candidate uses technical terminology such ...,4


In [47]:
# Calculate accuracy (exact match)
accuracy = (results['Predicted_Score'] == results['Actual_Score']).mean()

In [48]:
# Calculate Mean Absolute Error
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(results['Actual_Score'], results['Predicted_Score'])

print(f"Accuracy (exact match): {accuracy:.2f}")
print(f"Mean Absolute Error: {mae:.2f}")

Accuracy (exact match): 0.74
Mean Absolute Error: 0.27


In [49]:
results['Predicted_Score'].value_counts(), results['Actual_Score'].value_counts()

(Predicted_Score
 4    35
 3    29
 1    15
 2     9
 5     2
 Name: count, dtype: int64,
 Actual_Score
 4    39
 3    26
 1    14
 2     9
 5     2
 Name: count, dtype: int64)